In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
import numpy as np

from data_loader_pytorch import prepare_timit
from model_pytorch import SRCNN

In [2]:
batch_size = 64        # Number of samples in each minibatch
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 6, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

data_path = './serialized_train_data_cubic/'
data_loader = prepare_timit(data_path,
                            batch_size=batch_size,
                            shuffle=True,
                            extras=extras)

CUDA is supported


In [3]:
model = SRCNN()
model = model.to(computing_device)

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
def G_loss(y_true, y_pred):
    return torch.sqrt(torch.mean((y_true - y_pred) ** 2))

In [ ]:
epoch_number = 2
for epoch in enumerate(range(epoch_number)):
    
    t = tqdm_notebook(data_loader)
    for minibatch_count, (noisy, clean) in enumerate(t, 0):
        
        noisy = noisy.to(computing_device)
        clean = clean.to(computing_device)
        pred = model(noisy)
        
        optimizer.zero_grad()
        loss = G_loss(clean, pred)
        loss.backward()
        optimizer.step()
        
        t.set_description('loss:{:4f}'.format(loss))
        